In [1]:
def manual_lyrics_timing_correction(lyrics_with_timing, song_duration):
    """Allow user to manually correct lyrics timing with START TIME input"""
    
    print("\n" + "="*60)
    print("🎵 MANUAL LYRICS TIMING CORRECTION (START TIMES)")
    print("="*60)
    print("Current automatic timing:")
    print("-" * 50)
    
    # Display current timing
    for i, lyric in enumerate(lyrics_with_timing):
        print(f"{i+1:2d}. {lyric['start_time']:5.1f}s - {lyric['end_time']:5.1f}s: {lyric['text'][:40]}{'...' if len(lyric['text']) > 40 else ''}")
    
    print(f"\nSong duration: {song_duration:.1f}s")
    print("\nWould you like to manually adjust the START TIMES? (y/n): ")
    
    try:
        response = input().strip().lower()
        if response not in ['y', 'yes']:
            print("Using automatic timing...")
            return lyrics_with_timing
    except:
        print("Using automatic timing...")
        return lyrics_with_timing
    
    print("\n📝 MANUAL TIMING INSTRUCTIONS:")
    print("Enter timing in format: 'line_number,start_time'")
    print("Example: '1,5.5' means line 1 starts at 5.5 seconds")
    print("Example: '2,12.8' means line 2 starts at 12.8 seconds")
    print("Enter 'done' when finished, or 'skip' to use automatic timing")
    print("-" * 50)
    
    corrected_timing = lyrics_with_timing.copy()
    user_start_times = {}
    
    while True:
        try:
            user_input = input("\nEnter timing (line_number,start_time) or 'done': ").strip()
            
            if user_input.lower() in ['done', '']:
                break
            elif user_input.lower() == 'skip':
                print("Using automatic timing...")
                return lyrics_with_timing
            
            # Parse input
            parts = user_input.split(',')
            if len(parts) != 2:
                print("❌ Invalid format. Use: 'line_number,start_time'")
                continue
            
            line_num = int(parts[0].strip())
            start_time = float(parts[1].strip())
            
            # Validate line number
            if line_num < 1 or line_num > len(corrected_timing):
                print(f"❌ Line number must be between 1 and {len(corrected_timing)}")
                continue
            
            # Validate start time
            if start_time < 0 or start_time > song_duration:
                print(f"❌ Start time must be between 0 and {song_duration:.1f}")
                continue
            
            # Store user input
            user_start_times[line_num] = start_time
            line_index = line_num - 1
            
            print(f"✅ Set line {line_num} to start at {start_time:.1f}s: {corrected_timing[line_index]['text'][:30]}...")
            
            # Show current progress
            print("\nCurrent manual timing:")
            for i, lyric in enumerate(corrected_timing):
                marker = " ←" if (i+1) in user_start_times else ""
                user_start = user_start_times.get(i+1, lyric['start_time'])
                print(f"{i+1:2d}. {user_start:5.1f}s : {lyric['text'][:30]}...{marker}")
                
        except ValueError as e:
            print(f"❌ Invalid input: {e}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    # Apply all user corrections at once
    if user_start_times:
        corrected_timing = apply_start_time_corrections(corrected_timing, user_start_times, song_duration)
        print("\n✅ Manual timing correction completed!")
    else:
        print("\n⏩ No corrections made, using automatic timing...")
    
    return corrected_timing

In [2]:
def apply_start_time_corrections(lyrics_with_timing, user_start_times, song_duration):
    """Apply user-provided start times and calculate end times automatically"""
    
    corrected = []
    
    for i, lyric in enumerate(lyrics_with_timing):
        line_num = i + 1
        
        # Use user-provided start time if available, otherwise use automatic
        if line_num in user_start_times:
            start_time = user_start_times[line_num]
        else:
            start_time = lyric['start_time']
        
        # Calculate end time:
        # - If next line has user-defined start time, end there
        # - Otherwise, estimate based on line length or use next automatic timing
        if i < len(lyrics_with_timing) - 1:
            next_line_num = i + 2
            if next_line_num in user_start_times:
                end_time = user_start_times[next_line_num]
            else:
                # Estimate end time based on typical line duration (3-5 seconds)
                words = len(lyric['text'].split())
                estimated_duration = max(2.0, min(6.0, words * 0.8))  # 0.8 seconds per word
                end_time = min(start_time + estimated_duration, lyrics_with_timing[i+1]['start_time'])
        else:
            # Last line - end at song duration
            end_time = song_duration
        
        corrected.append({
            'text': lyric['text'],
            'start_time': start_time,
            'end_time': end_time
        })
    
    return corrected

In [3]:
def quick_start_time_correction(lyrics_with_timing, corrections_list, song_duration):
    """Apply quick manual corrections using a list of (line_number, start_time) tuples"""
    
    user_start_times = {}
    for line_num, start_time in corrections_list:
        user_start_times[line_num] = start_time
    
    return apply_start_time_corrections(lyrics_with_timing, user_start_times, song_duration)

In [4]:
def create_audio_synced_video_with_start_time_correction(song_id=songId, plot_analysis=True, max_duration=None, enable_manual_correction=True):
    """Create video with manual START TIME correction"""
    
    try:
        # Get song data
        query = f"""
        SELECT s.name as song_name, s.location as audio_file,
               l.content as lyrics, a.first_name, a.last_name
        FROM songs s 
        JOIN lyrics l ON s.id = l.song_id 
        JOIN artists a ON s.artist_id = a.id 
        WHERE s.id = {song_id}
        """
        
        df = pd.read_sql(query, engine)
        song_data = df.iloc[0]
        
        print(f"🎵 Creating AUDIO-SYNCED video for: {song_data['song_name']}")
        
        # Construct file paths
        audio_dir = os.path.join(r"C:\ruby\music\public\uploads\song\location", str(song_id))
        audio_path = os.path.join(audio_dir, song_data['audio_file'])
        background_image_path = os.path.join(audio_dir, "Folder.jpg")
        
        print(f"🔊 Audio: {os.path.basename(audio_path)}")
        print(f"🖼️ Background: {os.path.basename(background_image_path)}")
        
        if not os.path.exists(audio_path):
            print("❌ Audio file not found")
            return None
        
        # Perform audio analysis
        audio_analysis = detect_vocal_segments(audio_path, plot_analysis=plot_analysis)
        
        # Load audio clip
        audio_clip = AudioFileClip(audio_path)
        full_duration = audio_analysis.get('duration', audio_clip.duration)
        
        # Apply duration limit
        if max_duration:
            duration = min(max_duration, full_duration)
            print(f"⏱️ Using LIMITED duration: {duration:.1f}s (max_duration={max_duration}s)")
        else:
            duration = full_duration
            print(f"⏱️ Using FULL duration: {duration:.1f}s")
        
        # Trim audio if needed
        if max_duration and full_duration > max_duration:
            audio_clip = audio_clip.subclip(0, duration)
        
        # Assign lyrics to timing segments
        lyrics_with_timing = assign_lyrics_to_segments(song_data['lyrics'], audio_analysis)
        
        if not lyrics_with_timing:
            print("❌ Could not assign lyrics timing")
            return None
        
        # MANUAL START TIME CORRECTION STEP
        if enable_manual_correction:
            lyrics_with_timing = manual_lyrics_timing_correction(lyrics_with_timing, duration)
        
        # Filter lyrics to only include those within the limited duration
        if max_duration:
            lyrics_with_timing = [lyric for lyric in lyrics_with_timing if lyric['start_time'] < duration]
            # Adjust the end time of the last lyric to match the limited duration
            if lyrics_with_timing and lyrics_with_timing[-1]['end_time'] > duration:
                lyrics_with_timing[-1]['end_time'] = duration
        
        print(f"📝 Final timing: {len(lyrics_with_timing)} lyrics lines")
        print(f"⏱️ Video duration: {duration:.1f}s ({duration/60:.1f} minutes)")
        
        # Video settings
        fps = 24
        width, height = 640, 480
        
        def make_synced_frame(t):
            try:
                # Load background
                if os.path.exists(background_image_path):
                    bg_image = Image.open(background_image_path)
                    bg_image = bg_image.resize((width, height), Image.Resampling.LANCZOS)
                    frame = np.array(bg_image)
                else:
                    frame = np.full((height, width, 3), [40, 40, 80], dtype=np.uint8)
                
                # Convert to PIL for text drawing
                pil_img = Image.fromarray(frame)
                draw = ImageDraw.Draw(pil_img)
                
                # Load font
                try:
                    font = ImageFont.truetype("arial.ttf", 32)
                except:
                    try:
                        font = ImageFont.truetype("C:/Windows/Fonts/arial.ttf", 32)
                    except:
                        font = ImageFont.load_default()
                
                # Get current lyric based on timing
                current_line = get_current_lyric(t, lyrics_with_timing)
                
                if current_line:
                    # Calculate text position
                    try:
                        bbox = draw.textbbox((0, 0), current_line, font=font)
                    except AttributeError:
                        bbox = draw.textsize(current_line, font=font)
                        bbox = (0, 0, bbox[0], bbox[1])
                    
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]
                    x = (width - text_width) // 2
                    y = (height - text_height) // 2
                    
                    # Semi-transparent background for text
                    padding = 10
                    draw.rectangle([
                        x - padding, y - padding,
                        x + text_width + padding, y + text_height + padding
                    ], fill=(0, 0, 0, 180))
                    
                    # Text with shadow for readability
                    shadow_color = (0, 0, 0)
                    text_color = (255, 255, 255)
                    
                    # Shadow
                    draw.text((x+2, y+2), current_line, font=font, fill=shadow_color)
                    # Main text
                    draw.text((x, y), current_line, font=font, fill=text_color)
                
                return np.array(pil_img)
                
            except Exception as e:
                print(f"❌ Frame error at {t:.1f}s: {e}")
                return np.zeros((height, width, 3), dtype=np.uint8)
        
        # Create video
        print("🎬 Creating video with corrected timing...")
        video = VideoClip(make_synced_frame, duration=duration)
        video = video.set_audio(audio_clip)
        
        # Export
        output_dir = '../data/videos'
        os.makedirs(output_dir, exist_ok=True)
        
        # Include correction in filename
        correction_suffix = "_corrected" if enable_manual_correction else ""
        duration_suffix = f"_{max_duration}s" if max_duration else "_full"
        
        output_file = os.path.join(output_dir, f"{song_data['song_name']}_audio_synced{correction_suffix}{duration_suffix}.mp4")
        
        print("📹 Exporting video...")
        video.write_videofile(
            output_file, 
            fps=fps, 
            codec='libx264',
            audio_codec='aac',
            verbose=False,
            logger=None
        )
        
        print(f"✅ VIDEO CREATED: {output_file}")
        print(f"📊 File size: {os.path.getsize(output_file) / (1024*1024):.1f} MB")
        
        # Display final timing
        print("\n📋 FINAL Lyrics Timing (START TIMES):")
        print("-" * 50)
        for i, lyric in enumerate(lyrics_with_timing):
            print(f"{i+1:2d}. {lyric['start_time']:5.1f}s - {lyric['end_time']:5.1f}s: {lyric['text'][:40]}{'...' if len(lyric['text']) > 40 else ''}")
        
        # Clean up
        video.close()
        audio_clip.close()
        
        return output_file
        
    except Exception as e:
        print(f"❌ Video creation error: {e}")
        import traceback
        traceback.print_exc()
        return None

NameError: name 'songId' is not defined